# Surveying dust structure via GALEX MIS

## Learning goals

By the end of this tutorial you will be able to:

- Understand NUV and FUV images and how they are useful to study dust
- Navigate the GALEX MIS catalog
- Create a mosaic from several images

## Introduction

## Imports

## Additional Resources

For more information about the MAST archive and details about the tutorial: 

<a href="https://mast.stsci.edu/api/v0/index.html">MAST API</a> <br>
<a href="https://galex.stsci.edu/GR6/">GALEX via MAST</a> <br>

## About this Notebook

**Author**: Clara Puerto Sánchez <br>
**Updated on**: 01/08/2022

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>